This script processes GeoTIFFs to create Cloud Optimized GeoTIFFs (COGs). It includes the following steps:

1. Reprojects the input raster to EPSG:3857 (Web Mercator) for web visualization.
2. Converts single-band rasters to RGB using a custom colormap.
3. Adds an alpha band for transparency based on NoData values.
4. Generates overviews for efficient multi-resolution rendering.
5. Saves the output as a compressed, tiled, and optimized COG.

### import modules

In [ ]:
from osgeo import gdal
import numpy as np

### define the function

In [1]:
def create_rgb_cog_with_reprojection(input_tiff, output_cog):
    # Open the input GeoTIFF
    src_ds = gdal.Open(input_tiff, gdal.GA_ReadOnly)
    if not src_ds:
        raise ValueError(f"Unable to open input file: {input_tiff}")

    # Reproject to EPSG:3857
    print("Reprojecting raster to EPSG:3857...")
    temp_reprojected_tiff = "temp_reprojected.tif"
    gdal.Warp(
        temp_reprojected_tiff,
        src_ds,
        dstSRS="EPSG:3857",  # Target projection
        format="GTiff",
        dstNodata=0  # Ensure NoData value is set during reprojection
    )

    # Open the reprojected dataset
    reproj_ds = gdal.Open(temp_reprojected_tiff, gdal.GA_ReadOnly)

    if reproj_ds.RasterCount == 1:
        # Single-band grayscale raster
        print("Processing single-band raster...")
        band = reproj_ds.GetRasterBand(1)
        data = band.ReadAsArray()

        # Create RGB arrays
        r = np.zeros_like(data, dtype=np.uint8)
        g = np.zeros_like(data, dtype=np.uint8)
        b = np.zeros_like(data, dtype=np.uint8)

        # This section applies a custom colormap for single-band rasters. 
        # Adjust the color map values and corresponding RGB colors if your input GeoTIFF uses different classification values.
        color_map = {
            1: (255, 0, 0),    # Red
            2: (0, 255, 0),    # Green
            3: (0, 0, 255),    # Blue
            4: (255, 255, 0),  # Yellow
            5: (255, 165, 0)   # Orange
        }

        # Adjust this colormap to match classification values in your input GeoTIFF.
        # Add or modify keys to handle additional or different values.
        for value, color in color_map.items():
            r[data == value] = color[0]
            g[data == value] = color[1]
            b[data == value] = color[2]

        # Create an in-memory RGB raster
        driver = gdal.GetDriverByName("MEM")
        rgb_ds = driver.Create("", reproj_ds.RasterXSize, reproj_ds.RasterYSize, 4, gdal.GDT_Byte)  # 4 bands for RGBA
        rgb_ds.SetGeoTransform(reproj_ds.GetGeoTransform())
        rgb_ds.SetProjection(reproj_ds.GetProjection())
        rgb_ds.GetRasterBand(1).WriteArray(r)
        rgb_ds.GetRasterBand(2).WriteArray(g)
        rgb_ds.GetRasterBand(3).WriteArray(b)
        alpha = np.where(data == 0, 0, 255).astype(np.uint8)  # Create alpha band
        rgb_ds.GetRasterBand(4).WriteArray(alpha)
        rgb_ds.GetRasterBand(1).SetNoDataValue(0)
        rgb_ds.GetRasterBand(2).SetNoDataValue(0)
        rgb_ds.GetRasterBand(3).SetNoDataValue(0)
        rgb_ds.GetRasterBand(4).SetNoDataValue(0)

    elif reproj_ds.RasterCount == 3:
        # Already an RGB raster
        print("Input raster is already RGB.")
        driver = gdal.GetDriverByName("MEM")
        rgb_ds = driver.CreateCopy("", reproj_ds, 0)
        alpha_band = rgb_ds.GetRasterBand(1).ReadAsArray() > 0
        alpha = (alpha_band * 255).astype(np.uint8)  # Create alpha band for transparency
        rgb_ds.AddBand()
        rgb_ds.GetRasterBand(4).WriteArray(alpha)
        rgb_ds.GetRasterBand(4).SetNoDataValue(0)

    else:
        raise ValueError("Unsupported raster format: must be single-band or RGB.")

    # Generate overviews
    print("Generating overviews...")
    rgb_ds.BuildOverviews("NEAREST", [2, 4, 8, 16, 32])

    # Translate to Cloud Optimized GeoTIFF
    print("Creating Cloud Optimized GeoTIFF...")
    gdal.Translate(
        output_cog,
        rgb_ds,
        format="COG",
        creationOptions=[
            "COMPRESS=LZW",  # Compression
            "BIGTIFF=YES",   # Allow large files
        ],
    )

    print(f"RGB COG with reprojection and alpha band created successfully: {output_cog}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Reprojecting raster to EPSG:3857...


/Users/majew030/opt/anaconda3/envs/notebooks/lib/python3.10/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
ERROR 6: temp_reprojected.tif, band 1: SetColorTable() only supported for Byte or UInt16 bands in TIFF format.


Processing single-band raster...
Generating overviews...
Creating Cloud Optimized GeoTIFF...
RGB COG with reprojection and alpha band created successfully: aquifer_rgb_cog.tif


### Run the function

In [ ]:
create_rgb_cog_with_reprojection("Aquifer_Sensitivity_Near_Surface/Aquifer_Sensitivity_Near_Surface_IN.tif", "aquifer_rgb_cog.tif")